In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp core

In [ ]:
#export
import os
import random
from io import StringIO
from typing import Collection, Dict, Optional

import pandas as pd
import psycopg2
from dotenv import load_dotenv

Connection = psycopg2.extensions.connection

In [ ]:
#exporti
def _get_connection_arguments(server: str,
                              dotenv_path: str = '.env',
                              **conn_kwargs) -> Dict[str,str]:
    "Get connection arguments for server from .env file and conn_kwargs."
    load_dotenv(dotenv_path)
    variables = ('host', 'port', 'database', 'user', 'password')
    params = {variable: os.getenv(f'postgres_{variable}_{server}'.upper())
              for variable in variables}
    params.update(conn_kwargs)
    return params

In [ ]:
#export
def create_connection(server: str,
                      dotenv_path: str = '.env',
                      **conn_kwargs) -> Connection:
    "Create psycopg2 connection to server from .env file and conn_kwargs."
    conn_args = _get_connection_arguments(server, dotenv_path, **conn_kwargs)
    return psycopg2.connect(**conn_args)

You can create a connection by using the server name and specifying the path to the `.env` file.

```
con = create_connection('server', 'path/to/.env')
```

In [ ]:
#export
def append_dataframe_to_table(conn: Connection,
                              table: str,
                              df: pd.DataFrame) -> None:
    "Append a dataframe to an existing postgresql table."
    with conn.cursor() as cur:
        output = StringIO()
        df.to_csv(output, sep='\t', header=False, index=False)
        output.seek(0)
        cur.copy_from(output, table, null='')

This can be used with a connection context manager to execute the append in a transaction, like so:

```
with create_connection('server', 'path/to/.env') as conn:
    append_dataframe_to_table(conn, 'table_name', df)
```

If the append fails, a rollback is performed, else it is commited

In [ ]:
#export
def update_table_from_df(conn: Connection,
                         table: str,
                         df: pd.DataFrame,
                         join_cols: Collection[str],
                         update_cols: Optional[Collection[str]] = None,
                         extra_where: Optional[str] = None):
    """Updates a postgresql table using the contents of a dataframe.

    If update_cols is None (the default) join_cols are used in the where
    statement of the update and the remaining columns are updated,
    otherwise only update_cols are updated.

    extra_where is placed at the start of the where statement.
    """
    if update_cols is None:
        update_cols = df.columns.drop(join_cols)
    temp_table = f'temp_replacements_{random.randint(1, 1000):04}'
    create_query = f"""
        CREATE TEMP TABLE {temp_table}
        AS
        SELECT {', '.join(df.columns)}
        FROM {table}
        LIMIT 0
    """
    def _create_equals_statements(cols, join_str, left_prefix='old_table.'):
        statements = [f'{left_prefix}{col} = new_table.{col}' for col in cols]
        return join_str.join(statements)
    set_statement = _create_equals_statements(update_cols, join_str=',\n\t', left_prefix='')
    where_statement = _create_equals_statements(join_cols, join_str='\n\tAND ')
    if extra_where is not None:
        where_statement = extra_where + '\n\tAND ' + where_statement
    update_query = f"""
        UPDATE {table} AS old_table
        SET {set_statement}
        FROM {temp_table} new_table
        WHERE {where_statement}
    """
    with conn.cursor() as cur:
        cur.execute(create_query)
        append_dataframe_to_table(conn, temp_table, df)
        cur.execute(update_query)

This can be performed as well within a conext manager

```
with create_connection('server', 'path/to/.env') as conn:
    update_table_from_df(conn, 'table_name', df, join_cols='x')
```